In [1]:
from sklearn.linear_model import LogisticRegression
import pandas as pd 
import numpy as np 
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing.data import OneHotEncoder
from sklearn import metrics
import sys
import math
from sklearn.externals import joblib
from sklearn.metrics import roc_auc_score
import warnings
import math
warnings.filterwarnings("ignore")
from sklearn.model_selection import GridSearchCV
#
import pickle
def out_pickle(file_name,data):
    f = open(file_name,'wb')
    pickle.dump(data,f)
    f.close()

def in_pickle(file_name):
    f = open(file_name,'rb')
    data = pickle.load(f)
    f.close()
    return data
import gc
import xgboost as xgb

In [2]:
from sklearn.model_selection import train_test_split

In [8]:
pd.set_option('max_columns', 500)
pd.set_option('max_rows', 500)

In [7]:
feat = pd.read_csv('/home/odin/wuyifan/data/user_loan_demand/user_demand_model_event_stat_fea.csv',sep='\t')

In [8]:
feat.columns = [x.strip().split('.')[1] for x in feat.columns]
feat.columns

Index(['uid', 'funds_channel_id', 'server_date', 'label',
       'is_last_1d_market_biopen', 'last_1d_market_biopen_cnt',
       'is_last_3d_market_biopen', 'last_3d_market_biopen_cnt',
       'is_last_7d_market_biopen', 'last_7d_market_biopen_cnt',
       'is_last_1m_market_biopen', 'last_1m_market_biopen_cnt',
       'days_from_last_market_biopen', 'is_last_1d_market_setup_sw',
       'last_1d_market_setup_sw_cnt', 'is_last_3d_market_setup_sw',
       'last_3d_market_setup_sw_cnt', 'is_last_7d_market_setup_sw',
       'last_7d_market_setup_sw_cnt', 'is_last_1m_market_setup_sw',
       'last_1m_market_setup_sw_cnt', 'days_from_last_market_setup_sw',
       'is_last_1d_mycoupon_ck', 'last_1d_mycoupon_ck_cnt',
       'is_last_3d_mycoupon_ck', 'last_3d_mycoupon_ck_cnt',
       'is_last_7d_mycoupon_ck', 'last_7d_mycoupon_ck_cnt',
       'is_last_1m_mycoupon_ck', 'last_1m_mycoupon_ck_cnt',
       'days_from_last_mycoupon_ck', 'is_last_1d_market_loan_list_sw',
       'last_1d_market_loan_li

In [10]:
loan = pd.read_csv('/home/odin/lijunwei/yiyuan/feat_loan.tsv',sep='\t',nrows=1000000)
loan.columns = [x.strip().split('.')[1] for x in loan.columns]
loan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Columns: 457 entries, uid to current_valid_coupon_num
dtypes: float64(414), int64(4), object(39)
memory usage: 3.4+ GB


In [20]:
loan.count().sort_values(ascending=False)

uid                                                        3244906
dsd_market_biopen_tag                                      3244906
last_week_align                                            3244906
label                                                      3244906
dsd_loan_confirm_button_ck_tag                             3244906
server_date                                                3244906
last_3_month_drip_repayment_result_page_num                3161864
last_6_month_drip_repayment_button_num                     3161864
last_30_day_drip_help_num                                  3161864
last_14_day_drip_help_num                                  3161864
last_7_day_drip_help_num                                   3161864
his_drip_repayment_button_num                              3161864
last_1_year_drip_repayment_button_num                      3161864
last_3_month_drip_repayment_button_num                     3161864
last_6_month_drip_help_num                                 316

In [25]:
loan.dtypes

NameError: name 'loan' is not defined

In [11]:
data = loan.loc[:,(loan.dtypes=='int64') | (loan.dtypes=='float64')]
data = pd.concat([data,loan['server_date']],axis=1)
del loan
gc.collect()

76

In [ ]:
# chuxing2 = pd.read_csv('/home/odi/lijunwei/yiyuan/feat_chuxing2.tsv',sep='\t')
# chuxing2.columns = chuxing2.columns.strip().split()[1]
# chuxing2.info()

In [9]:
data = feat.copy()
del feat
gc.collect()
default_value=-999
data = data.fillna(default_value)

In [11]:
train_index = data[(data['server_date']<='2020-07-20')].index
test_index = data[(data['server_date']>'2020-07-20')].index
print(len(train_index),len(test_index))
train_label = data.loc[train_index, 'label']
test_label = data.loc[test_index, 'label']

data = data.drop(['uid','label','server_date'],axis = 1)

D_train = xgb.DMatrix(data.loc[train_index, :], label=train_label)
D_test = xgb.DMatrix(data.loc[test_index, :], label=test_label)

del data
gc.collect()

ntrees = 1000
param = {'max_depth':3, 'eta':0.1, 'silent':1, 'objective':'binary:logistic', 
         'eval_metric':'auc', 'scale_pos_weight':1, 
         'subsample':0.8, 'seed':41,'colsample_bytree':0.8, 'lambda':10,'base_score':0.5, 'verbose_eval':10, 'nthread':10}
watchlist = [(D_train,'train'), (D_test, 'eval')]
bst_whole_tmp = xgb.train(param, D_train, ntrees, watchlist, early_stopping_rounds=20)

2205498 549483
[0]	train-auc:0.703124	eval-auc:0.700784
Multiple eval metrics have been passed: 'eval-auc' will be used for early stopping.

Will train until eval-auc hasn't improved in 20 rounds.
[1]	train-auc:0.713063	eval-auc:0.709063
[2]	train-auc:0.714043	eval-auc:0.709033
[3]	train-auc:0.715977	eval-auc:0.711551
[4]	train-auc:0.721326	eval-auc:0.715114
[5]	train-auc:0.723701	eval-auc:0.717304
[6]	train-auc:0.7248	eval-auc:0.718669
[7]	train-auc:0.725733	eval-auc:0.719636
[8]	train-auc:0.726213	eval-auc:0.719952
[9]	train-auc:0.726997	eval-auc:0.720513
[10]	train-auc:0.727441	eval-auc:0.720614
[11]	train-auc:0.728257	eval-auc:0.721394
[12]	train-auc:0.728746	eval-auc:0.72187
[13]	train-auc:0.729077	eval-auc:0.722031
[14]	train-auc:0.73047	eval-auc:0.723525
[15]	train-auc:0.731535	eval-auc:0.724651
[16]	train-auc:0.73215	eval-auc:0.725433
[17]	train-auc:0.732671	eval-auc:0.726023
[18]	train-auc:0.733251	eval-auc:0.726644
[19]	train-auc:0.733646	eval-auc:0.727021
[20]	train-auc:0.73

In [12]:
default_value=-999
data = data.fillna(default_value)

In [23]:
data.head(100)

,uid,funds_channel_id,dsd_market_biopen_tag,dsd_loan_confirm_button_ck_tag,label,current_drip_credit_line,current_drip_interest_rate,his_drip_max_credit_line,his_drip_min_credit_line,last_7_day_drip_weekend_apply_loan_num,last_14_day_drip_weekend_apply_loan_num,last_30_day_drip_weekend_apply_loan_num,last_3_month_drip_weekend_apply_loan_num,last_6_month_drip_weekend_apply_loan_num,last_1_year_drip_weekend_apply_loan_num,last_30_day_drip_night_apply_loan_num,last_3_month_drip_night_apply_loan_num,last_6_month_drip_night_apply_loan_num,last_1_year_drip_night_apply_loan_num,last_7_day_drip_night_apply_loan_num,last_14_day_drip_night_apply_loan_num,last_7_day_drip_apply_loan_num,last_14_day_drip_apply_loan_num,last_30_day_drip_apply_loan_num,last_3_month_drip_apply_loan_num,last_6_month_drip_apply_loan_num,last_1_year_drip_apply_loan_num,his_drip_apply_loan_num,his_drip_apply_loans_success_num,current_drip_credit_line_balance,current_drip_credit_line_use_rate,last_7_day_drip_apply_loan_funds_channel_num,last_14_day_drip_apply_loan_funds_channel_num,last_30_day_drip_apply_loan_funds_channel_num,last_3_month_drip_apply_loan_funds_channel_num,last_6_month_drip_apply_loan_funds_channel_num,last_1_year_drip_apply_loan_funds_channel_num,total_drip_apply_loan_funds_channel_num,last_7_day_drip_apply_loan_suc_funds_channel_num,last_14_day_drip_apply_loan_suc_funds_channel_num,last_30_day_drip_apply_loan_suc_funds_channel_num,last_3_month_drip_apply_loan_suc_funds_channel_num,last_6_month_drip_apply_loan_suc_funds_channel_num,last_1_year_drip_apply_loan_suc_funds_channel_num,total_drip_apply_loan_suc_funds_channel_num,last_drip_funds_channel_id,drip_user_role,last_credit_identity_card_city_id,last_credit_identity_card_user_age,last_credit_identity_card_user_sex,last_credit_mobile_city_id,first_drip_disburse_succ_principal,last_drip_isburse_succ_principal,first_drip_disburse_succ_duration,last_7_day_drip_loan_num,last_14_day_drip_loan_num,last_30_day_drip_loan_num,last_3_month_drip_loan_num,last_6_month_drip_loan_num,last_1_year_drip_loan_num,last_7_day_drip_loan_money,last_14_day_drip_loan_money,last_30_day_drip_loan_money,last_3_month_drip_loan_money,last_6_month_drip_loan_money,last_1_year_drip_loan_money,last_7_day_drip_weekend_loan_num,last_14_day_drip_weekend_loan_num,last_30_day_drip_weekend_loan_num,last_3_month_drip_weekend_loan_num,last_6_month_drip_weekend_loan_num,last_1_year_drip_weekend_loan_num,last_7_day_drip_weekend_loan_money,last_14_day_drip_weekend_loan_money,last_30_day_drip_weekend_loan_money,last_3_month_drip_weekend_loan_money,last_6_month_drip_weekend_loan_money,last_1_year_drip_weekend_loan_money,last_7_day_drip_loan_failed_num,last_14_day_drip_loan_failed_num,last_30_day_drip_loan_failed_num,last_3_month_drip_loan_failed_num,last_6_month_drip_loan_failed_num,last_1_year_drip_loan_failed_num,last_7_day_drip_loan_failed_money,last_14_day_drip_loan_failed_money,last_30_day_drip_loan_failed_money,last_3_month_drip_loan_failed_money,last_6_month_drip_loan_failed_money,last_1_year_drip_loan_failed_money,last_7_day_drip_loan_service_fee,last_14_day_drip_loan_service_fee,last_30_day_drip_loan_service_fee,last_3_month_drip_loan_service_fee,last_6_month_drip_loan_service_fee,last_1_year_drip_loan_service_fee,his_drip_loan_num,his_drip_loan_money,last_7_day_drip_night_loan_num,last_14_day_drip_night_loan_num,last_30_day_drip_night_loan_num,last_3_month_drip_night_loan_num,last_6_month_drip_night_loan_num,last_1_year_drip_night_loan_num,last_7_day_drip_night_loan_money,last_14_day_drip_night_loan_money,last_30_day_drip_night_loan_money,last_3_month_drip_night_loan_money,last_6_month_drip_night_loan_money,last_1_year_drip_night_loan_money,first_disburse,first_drip_disburse_total_duration,his_drip_total_duration,avg_drip_mon_loan_num,avg_drip_mon_loan_money,max_drip_total_duration,last_7_day_drip_max_loan_total_duration,last_14_day_drip_max_loan_total_duration,last_30_day_drip_max_loan_total_duration,last_3_month_drip_max_loan_t

In [17]:
y = data['label']
x = data.drop(['uid','dsd_market_biopen_tag','dsd_loan_confirm_button_ck_tag','label','server_date'],axis = 1)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3,random_state=0)
columns = data.columns
del x,y,data
gc.collect()
D_train = xgb.DMatrix(x_train, label=y_train)
D_test = xgb.DMatrix(x_test, label=y_test)

ntrees = 1000
param = {'max_depth':3, 'eta':0.1, 'silent':1, 'objective':'binary:logistic', 
         'eval_metric':'auc', 'scale_pos_weight':1, 
         'subsample':0.8, 'seed':41,'colsample_bytree':0.8, 'lambda':10,'base_score':0.5, 'verbose_eval':10, 'nthread':10}
watchlist = [(D_train,'train'), (D_test, 'eval')]
bst_whole_tmp = xgb.train(param, D_train, ntrees, watchlist, early_stopping_rounds=20)

58

In [13]:
train_index = data[(data['server_date']<='2020-07-20')].index
test_index = data[(data['server_date']>'2020-07-20')].index
print(len(train_index),len(test_index))
train_label = data.loc[train_index, 'label']
test_label = data.loc[test_index, 'label']

print(len(data.columns))
data = data.drop(['uid','dsd_market_biopen_tag','dsd_loan_confirm_button_ck_tag','label','server_date'],axis = 1)
print(len(data.columns))
columns = data.columns

D_train = xgb.DMatrix(data.loc[train_index, :], label=train_label)
D_test = xgb.DMatrix(data.loc[test_index, :], label=test_label)

del data
gc.collect()

ntrees = 1000
param = {'max_depth':3, 'eta':0.1, 'silent':1, 'objective':'binary:logistic', 
         'eval_metric':'auc', 'scale_pos_weight':1, 
         'subsample':0.8, 'seed':41,'colsample_bytree':0.8, 'lambda':10,'base_score':0.5, 'verbose_eval':10, 'nthread':10}
watchlist = [(D_train,'train'), (D_test, 'eval')]
bst_whole_tmp = xgb.train(param, D_train, ntrees, watchlist, early_stopping_rounds=20)

795519 204481
419
414
[0]	train-auc:0.702511	eval-auc:0.696144
Multiple eval metrics have been passed: 'eval-auc' will be used for early stopping.

Will train until eval-auc hasn't improved in 20 rounds.
[1]	train-auc:0.734323	eval-auc:0.724953
[2]	train-auc:0.736059	eval-auc:0.726674
[3]	train-auc:0.744433	eval-auc:0.734873
[4]	train-auc:0.75289	eval-auc:0.742963
[5]	train-auc:0.756836	eval-auc:0.746994
[6]	train-auc:0.756862	eval-auc:0.746972
[7]	train-auc:0.758933	eval-auc:0.749033
[8]	train-auc:0.761527	eval-auc:0.751514
[9]	train-auc:0.761417	eval-auc:0.751244
[10]	train-auc:0.763276	eval-auc:0.753183
[11]	train-auc:0.765499	eval-auc:0.755485
[12]	train-auc:0.766654	eval-auc:0.756485
[13]	train-auc:0.766847	eval-auc:0.756586
[14]	train-auc:0.767605	eval-auc:0.75721
[15]	train-auc:0.767825	eval-auc:0.757543
[16]	train-auc:0.768586	eval-auc:0.758418
[17]	train-auc:0.769209	eval-auc:0.759235
[18]	train-auc:0.769361	eval-auc:0.759341
[19]	train-auc:0.770495	eval-auc:0.760531
[20]	trai

In [19]:
ntrees = 1000
param = {'max_depth':3, 'eta':0.1, 'silent':1, 'objective':'binary:logistic', 
         'eval_metric':'auc', 'scale_pos_weight':1, 
         'subsample':0.8, 'seed':41,'colsample_bytree':0.8, 'lambda':10,'base_score':0.5, 'verbose_eval':10, 'nthread':10}
watchlist = [(D_train,'train'), (D_test, 'eval')]
bst_whole_tmp = xgb.train(param, D_train, ntrees, watchlist, early_stopping_rounds=20)

[0]	train-auc:0.704826	eval-auc:0.705715
Multiple eval metrics have been passed: 'eval-auc' will be used for early stopping.

Will train until eval-auc hasn't improved in 20 rounds.
[1]	train-auc:0.731709	eval-auc:0.732542
[2]	train-auc:0.739872	eval-auc:0.741266
[3]	train-auc:0.746097	eval-auc:0.747554
[4]	train-auc:0.745831	eval-auc:0.747334
[5]	train-auc:0.74797	eval-auc:0.749592
[6]	train-auc:0.752598	eval-auc:0.75422
[7]	train-auc:0.754801	eval-auc:0.756339
[8]	train-auc:0.758253	eval-auc:0.759881
[9]	train-auc:0.759891	eval-auc:0.761393
[10]	train-auc:0.761373	eval-auc:0.763206
[11]	train-auc:0.762045	eval-auc:0.763923
[12]	train-auc:0.763239	eval-auc:0.765176
[13]	train-auc:0.763897	eval-auc:0.765943
[14]	train-auc:0.763628	eval-auc:0.765644
[15]	train-auc:0.764369	eval-auc:0.766314
[16]	train-auc:0.765601	eval-auc:0.767425
[17]	train-auc:0.765846	eval-auc:0.767693
[18]	train-auc:0.766278	eval-auc:0.76812
[19]	train-auc:0.767228	eval-auc:0.769032
[20]	train-auc:0.767818	eval-auc

In [14]:
def ks(y_true, y_pred):
    fpr,tpr,thresholds= metrics.roc_curve(y_true,y_pred)
    return max(tpr-fpr)
def print_result(y_score, y_test, thr=0.5):
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_score, pos_label = 1)
    print('auc:{}'.format(metrics.auc(fpr, tpr)))

print("evaluate on test data:")
y_score_test = bst_whole_tmp.predict(D_test, ntree_limit=1000)
print_result(y_score_test, D_test.get_label()), ks(D_test.get_label(),y_score_test)

evaluate on test data:
auc:0.8059134747291841


(None, 0.4515146746356506)

In [ ]:
bst_whole_tmp.save_model('./all_in_one_tmp_fd.model')

In [15]:
import operator
def print_feature_importance(feature, bst, method='weight'):
    top_fea = []
    top_col=[]
    n=0
    if method == 'weight':
        fs = bst.get_fscore()
        all_features = [fs.get(f, 0.) for f in bst.feature_names]
        all_features = np.array(all_features, dtype=np.float32)
        feature_importance = all_features / all_features.sum()
        feature_combine = sorted(zip(feature, feature_importance), key=lambda x: x[1], reverse=True)
        for ff, imp in feature_combine:
            if imp > 0:
                print(ff, imp)
                top_fea.append([ff,imp])
                top_col.append(ff)
            n += 1
    elif method == 'gain':
        importance = bst.get_score(importance_type='gain');
        importance = sorted(importance.items(), key=operator.itemgetter(1), reverse=True);
        for ff, imp in importance:
            print(ff, imp)
            top_fea.append([ff,imp])
            n +=1 
#     print(n)
    return top_fea, top_col

In [16]:
fea_imp, fea_cols = print_feature_importance(columns, bst_whole_tmp, method='weight')

current_drip_credit_line_balance 0.05966519
current_drip_credit_line_use_rate 0.030476462
last_3_month_drip_num 0.021748463
his_face_num 0.02017456
last_7_day_drip_num 0.018457577
last_6_month_drip_num 0.017026756
his_drip_num 0.01445128
his_drip_repay_not_done_num 0.012591215
avg_drip_total_money 0.011732723
last_14_day_drip_repay_money 0.011732723
last_30_day_drip_num 0.011732723
his_drip_loan_result_page_num 0.011303477
last_drip_isburse_succ_principal 0.010444985
last_1_year_drip_num 0.010444985
his_drip_max_credit_line 0.010301903
last_3_month_drip_loan_result_page_num 0.010301903
current_drip_in_loan_money 0.009872657
last_6_month_drip_loan_result_page_num 0.009729575
last_3_month_drip_loan_failed_num 0.009443411
last_drip_funds_channel_id 0.009300329
his_drip_homepage_num 0.009300329
current_drip_interest_rate 0.009014165
last_30_day_drip_loan_failed_num 0.008728001
avg_drip_mon_loan_money 0.008728001
last_30_day_drip_repay_money 0.0084418375
last_7_day_drip_repay_money 0.008298

In [17]:
fea_imp_gain, fea_cols_gain = print_feature_importance(columns, bst_whole_tmp, method='gain')

last_14_day_drip_loan_failed_num 1813.2633778625
last_14_day_drip_loan_failed_money 1442.6572723611107
last_30_day_drip_loan_failed_num 1408.9326636557375
today_drip_overdue_money 669.8184765238461
his_drip_repay_not_done_num 537.9919864053408
is_drip_account_balanced 494.3410330170832
his_drip_max_credit_line 490.47323199374995
last_14_day_drip_apply_loan_suc_funds_channel_num 470.28901443333336
last_30_day_drip_loan_failed_money 463.54009967447354
current_drip_credit_line_use_rate 379.00191927718316
current_drip_in_loan_num 359.51991652500004
last_7_day_drip_loan_failed_num 353.4567867942856
current_drip_credit_line_balance 335.13542593474807
last_14_day_drip_apply_loan_num 329.15173093333334
last_1_year_drip_loan_failed_num 304.75981611823534
last_3_month_drip_loan_num 300.37632238452824
last_7_day_drip_repay_principal 294.95124719
current_drip_in_loan_money 277.42147859231886
last_30_day_drip_loan_result_page_num 273.47329042473694
last_14_day_drip_loan_result_page_num 263.82397938

In [18]:
fea_imp[:50]

[['current_drip_credit_line_balance', 0.05966519],
 ['current_drip_credit_line_use_rate', 0.030476462],
 ['last_3_month_drip_num', 0.021748463],
 ['his_face_num', 0.02017456],
 ['last_7_day_drip_num', 0.018457577],
 ['last_6_month_drip_num', 0.017026756],
 ['his_drip_num', 0.01445128],
 ['his_drip_repay_not_done_num', 0.012591215],
 ['avg_drip_total_money', 0.011732723],
 ['last_14_day_drip_repay_money', 0.011732723],
 ['last_30_day_drip_num', 0.011732723],
 ['his_drip_loan_result_page_num', 0.011303477],
 ['last_drip_isburse_succ_principal', 0.010444985],
 ['last_1_year_drip_num', 0.010444985],
 ['his_drip_max_credit_line', 0.010301903],
 ['last_3_month_drip_loan_result_page_num', 0.010301903],
 ['current_drip_in_loan_money', 0.009872657],
 ['last_6_month_drip_loan_result_page_num', 0.009729575],
 ['last_3_month_drip_loan_failed_num', 0.009443411],
 ['last_drip_funds_channel_id', 0.009300329],
 ['his_drip_homepage_num', 0.009300329],
 ['current_drip_interest_rate', 0.009014165],
 ['la

In [19]:
fea_imp_gain[:50]

[['last_14_day_drip_loan_failed_num', 1813.2633778625],
 ['last_14_day_drip_loan_failed_money', 1442.6572723611107],
 ['last_30_day_drip_loan_failed_num', 1408.9326636557375],
 ['today_drip_overdue_money', 669.8184765238461],
 ['his_drip_repay_not_done_num', 537.9919864053408],
 ['is_drip_account_balanced', 494.3410330170832],
 ['his_drip_max_credit_line', 490.47323199374995],
 ['last_14_day_drip_apply_loan_suc_funds_channel_num', 470.28901443333336],
 ['last_30_day_drip_loan_failed_money', 463.54009967447354],
 ['current_drip_credit_line_use_rate', 379.00191927718316],
 ['current_drip_in_loan_num', 359.51991652500004],
 ['last_7_day_drip_loan_failed_num', 353.4567867942856],
 ['current_drip_credit_line_balance', 335.13542593474807],
 ['last_14_day_drip_apply_loan_num', 329.15173093333334],
 ['last_1_year_drip_loan_failed_num', 304.75981611823534],
 ['last_3_month_drip_loan_num', 300.37632238452824],
 ['last_7_day_drip_repay_principal', 294.95124719],
 ['current_drip_in_loan_money', 27